In [1]:
import pandas as pd
import statsmodels.api as sm
from linearmodels.iv import IV2SLS

In [2]:
url = 'http://qcpages.qc.cuny.edu/~rvesselinov/statadata/OPENNESS.DTA'
df = pd.read_stata(url)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114 entries, 0 to 113
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   open     114 non-null    float32
 1   inf      114 non-null    float32
 2   pcinc    114 non-null    int16  
 3   land     114 non-null    float32
 4   oil      114 non-null    int8   
 5   good     114 non-null    int8   
 6   lpcinc   114 non-null    float32
 7   lland    114 non-null    float32
 8   lopen    114 non-null    float32
 9   linf     114 non-null    float32
 10  opendec  114 non-null    float32
 11  linfdec  114 non-null    float32
dtypes: float32(9), int16(1), int8(2)
memory usage: 5.3 KB


In [4]:
# Estimation par MCO de la forme réduite(équation de "open")

open_mco = sm.OLS(df['open'], sm.add_constant(df[['pcinc', 'lland']]))
open_res = open_mco.fit(cov_type='HC0')
print(open_res.summary()) 

                            OLS Regression Results                            
Dep. Variable:                   open   R-squared:                       0.449
Model:                            OLS   Adj. R-squared:                  0.439
Method:                 Least Squares   F-statistic:                     22.49
Date:                Mon, 06 Mar 2023   Prob (F-statistic):           6.30e-09
Time:                        11:36:35   Log-Likelihood:                -488.45
No. Observations:                 114   AIC:                             982.9
Df Residuals:                     111   BIC:                             991.1
Df Model:                           2                                         
Covariance Type:                  HC0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        122.7678     13.906      8.828      0.0

In [5]:
# 2MC

model_2mc = IV2SLS(df['inf'], sm.add_constant(df['pcinc']), df['open'], df['lland'])
res_2mc = model_2mc.fit(cov_type="robust")
print(res_2mc)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                    inf   R-squared:                      0.0407
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0234
No. Observations:                 114   F-statistic:                    5.8614
Date:                Mon, Mar 06 2023   P-value (F-stat)                0.0534
Time:                        11:37:12   Distribution:                  chi2(2)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          30.690     7.3212     4.1919     0.0000      16.340      45.039
pcinc         -0.0004     0.0003    -1.5362     0.12

In [18]:
# Comparaison MCO.

model_mco = sm.OLS(df['inf'], sm.add_constant(df[['pcinc', 'open']]))
res_mco = model_mco.fit(cov_type='HC0')
print(res_mco.summary())


                            OLS Regression Results                            
Dep. Variable:                    inf   R-squared:                       0.052
Model:                            OLS   Adj. R-squared:                  0.035
Method:                 Least Squares   F-statistic:                     4.159
Date:                Mon, 14 Mar 2022   Prob (F-statistic):             0.0181
Time:                        14:40:37   Log-Likelihood:                -520.49
No. Observations:                 114   AIC:                             1047.
Df Residuals:                     111   BIC:                             1055.
Df Model:                           2                                         
Covariance Type:                  HC0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         26.8379      5.098      5.264      0.0

In [19]:
df['open_sq'] = df.open ** 2
df['lland_sq'] = df.lland ** 2

In [21]:
model_2mc = IV2SLS(df['inf'], sm.add_constant(df['pcinc']), df[['open', 'open_sq']], df[['lland', 'lland_sq']])
res_2mc = model_2mc.fit(cov_type="robust")
print(res_2mc)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                    inf   R-squared:                     -0.0153
Estimator:                    IV-2SLS   Adj. R-squared:                -0.0430
No. Observations:                 114   F-statistic:                    5.1894
Date:                Mon, Mar 14 2022   P-value (F-stat)                0.1584
Time:                        14:42:31   Distribution:                  chi2(3)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          49.014     17.544     2.7937     0.0052      14.628      83.399
pcinc         -0.0004     0.0003    -1.2036     0.22